In [1]:
%%time
import sys
import re
import math
import requests
import enlighten
import sqlite3

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

url = 'http://200.152.38.155/CNPJ/'
formato_data = '%Y%m%d'
PATH_BD = 'data/CNPJ_full.db'

CPU times: total: 141 ms
Wall time: 873 ms


In [9]:
%%time
list_urls = re.findall(r'href=[\'"]?([^\'" >]+)', requests.get(url).text)
filtered = list(filter(lambda url: url.split(".")[-1] in ["zip"], list_urls))
tamanho = len(filtered)

CPU times: total: 0 ns
Wall time: 1.9 s


In [10]:
%%time
MANAGER = enlighten.get_manager()
bara_lista = MANAGER.counter(total=tamanho, desc='Processo', unit='Arquivos')

print("")
print(f"Fazendo download dos {tamanho} arquivos encontrados.", flush=False)
print("")


for indice, elemento in enumerate(filtered):
    bara_lista.update()
    
    r = requests.get(url + elemento, stream=True, headers={"Accept-Encoding": None})
    if r.status_code == requests.codes.ok:
        assert r.status_code == 200, r.status_code
        dlen = int(r.headers.get('Content-Length', '0')) or None
        barra_download = MANAGER.counter(color = 'green', desc= f'Arquivo {indice + 1} de {tamanho}', 
                             total = dlen and math.ceil(dlen / 2 ** 20), 
                             unit = 'MiB', leave = False)
        with barra_download as ctr, \
             open("zips/" + elemento, 'wb', buffering = 2 ** 20) as f:
            for chunk in r.iter_content(chunk_size = 2 ** 20):
                # print(chunk[-16:].hex().upper())
                print(f'\r{elemento} ------ Está sendo baixado!!!', end='', flush=True)
                sys.stdout.flush()
                f.write(chunk)
                ctr.update()
        
    else:
        r.raise_for_status()
print(f'\r Download Concluido!!!', end='', flush=True)
MANAGER.stop()


Fazendo download dos 37 arquivos encontrados.

 Download Concluido!!!á sendo baixado!!!ado!!!o!!!CPU times: total: 23.4 s
Wall time: 2h 3min 30s


In [11]:
%%time
dict_aux = {
    'Simples': [
        'DADOS_SIMPLES',
        'CREATE TABLE IF NOT EXISTS DADOS_SIMPLES (\n  CNPJ               VARCHAR (8) PRIMARY KEY,\n    OPCAO_SIMPLES      VARCHAR (1),\n    DATA_OPCAO_SIMPLES DATE,\n    DATA_EXCLUSAO      DATE,\n    OPCAO_MEI          VARCHAR (1),\n    DATA_OPCAO_MEI     DATE,\n    DATA_EXCLUSAO_MEI  DATE\n)',
        ['cnpj','opcao_simples','data_opcao_simples','data_exclusao','opcao_mei','data_opcao_mei','data_exclusao_mei'],
        {'cnpj': str}
        ],
    'Cnaes': [
        'CNAES',
        'CREATE TABLE IF NOT EXISTS CNAES (\n    CODIGO    VARCHAR PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'Motivos': [
        'MOTIVOS',
        'CREATE TABLE IF NOT EXISTS MOTIVOS (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'Municipios': [
        'MUNICIPIOS',
        'CREATE TABLE IF NOT EXISTS MUNICIPIOS (\n    CODIGO    INTEGERL PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'Naturezas': [
        'NATUREZAS',
        'CREATE TABLE IF NOT EXISTS NATUREZAS (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'Paises': [
        'PAISES',
        'CREATE TABLE IF NOT EXISTS PAISES (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'Qualificacoes': [
        'QUALIFICACOES',
        'CREATE TABLE IF NOT EXISTS QUALIFICACOES (\n    CODIGO    INTEGER PRIMARY KEY,\n    DESCRICAO VARCHAR\n)',
        ['codigo','descricao'],
        {'descricao': str}
        ],
     'Empresas': [
        'EMPRESAS',
        'CREATE TABLE IF NOT EXISTS EMPRESAS (\n    CNPJ            VARCHAR (8),\n    RAZAO           VARCHAR,\n    NATUREZA        INTEGER,\n    QUALIFICACAO_PF INTEGER,\n    CAPITAL         FLOAT,\n    PORTE           INTEGER,\n    ENTE_FEDERATIVO INTEGER\n)',
        ['cnpj','razao','natureza','qualificacao_pf','capital','porte','ente_federativo'],
        {'cnpj': str}
        ],
    'Estabelecimentos': [
        'ESTABELECIMENTOS',
        'CREATE TABLE IF NOT EXISTS ESTABELECIMENTOS \n                    (CNPJ                   VARCHAR (8),\n                    CNPJ_ORDEM             VARCHAR (4),\n                    CNPJ_DV                VARCHAR (2),\n                    MATRIZ_FILIAL          INTEGER,\n                    NOME                   VARCHAR,\n                    SITUACAO               INTEGER,\n                    DATA_SITUACAO          DATE,\n                    MOTIVO_SITUACAO        VARCHAR,\n                    CIDADE_EXTERIOR        VARCHAR,\n                    PAIS                   INTEGER,\n                    INICIO_ATIVIDADE       DATE,\n                    CNAE_FISCAL            VARCHAR,\n                    CNAE_SECUNDARIO        VARCHAR,\n                    TIPO_LOGRADOURO        VARCHAR,\n                    LOGRADOURO             VARCHAR,\n                    NUMERO                 VARCHAR,\n                    COMPLEMENTO            VARCHAR,\n                    BAIRRO                 VARCHAR,\n                    CEP                    VARCHAR (8),\n                    UF                     VARCHAR (2),\n                    MUNICIPIO              INTEGER,\n                    DDD_1                  VARCHAR,\n                    TELEFONE_1             VARCHAR (11),\n                    DDD_2                  INTEGER,\n                    TELEFONE_2             VARCHAR (11),\n                    DDD_FAX                INTEGER,\n                    NUMERO_FAX             VARCHAR (11),\n                    EMAIL                  VARCHAR,\n                    SITUACAO_ESPECIAL      VARCHAR,\n                    DATA_SITUACAO_ESPECIAL DATE \n                    )',
        ['cnpj','cnpj_ordem','cnpj_dv','matriz_filial','nome','situacao','data_situacao','motivo_situacao','cidade_exterior','pais','inicio_atividade','cnae_fiscal','cnae_secundario','tipo_logradouro','logradouro','numero','complemento','bairro','cep','uf','municipio','ddd_1','telefone_1','ddd_2','telefone_2','ddd_fax','numero_fax','email','situacao_especial','data_situacao_especial'],
        {'cnpj': str, 'cnpj_ordem': str, 'cnpj_dv': str, 'MOTIVO_SITUACAO': int}
        ],
    'Socios': [
        'SOCIOS',
        'CREATE TABLE IF NOT EXISTS SOCIOS (\n    CNPJ                             VARCHAR (8) ,\n    IDENTIFICADOR_SOCIO              INTEGER,\n    NOME                             VARCHAR,\n    CPF_CNPJ                         VARCHAR (14),\n    QUALIFICACAO                     INTEGER,\n    DATA_ENTRADA_SOCIEDADE           DATE,\n    CODIGO_PAIS                      INTEGER,\n    CPF_REPRESENTANTE_LEGAL          VARCHAR (11),\n    NOME_REPRESENTANTE_LEGAL         VARCHAR,\n    QUALIFICACAO_REPRESENTANTE_LEGAL INTEGER,\n    FAIXA_ETARIA                     INTEGER\n)',
        ['cnpj','identificador_socio','nome','cpf_cnpj','qualificacao','data_entrada_sociedade','codigo_pais','cpf_representante_legal','nome_representante_legal','qualificacao_representante_legal','faixa_etaria'],
        {'cnpj': str}
        ]    
    }

CPU times: total: 0 ns
Wall time: 999 µs


In [12]:
%%time
connection = sqlite3.connect(PATH_BD)
con = connection.cursor()

CPU times: total: 0 ns
Wall time: 12 ms


In [13]:
%%time
manager = enlighten.get_manager()
status_bar = manager.status_bar('Convertendo...',
                            color='blue',
                            justify=enlighten.Justify.CENTER)
pbar = enlighten.Counter(total=len(dict_aux), desc='Tabela:' , unit='Tabelas',color='green')
pbar.refresh()
for key, value in dict_aux.items():       
    con.execute('DROP TABLE IF EXISTS ' + value[0]) 
    con.execute(value[1])
    
    lista = [l for l in filtered if key in l]
    pbar1 = enlighten.Counter(total=len(lista), desc='Aquivo CSV: ' + value[0] , unit='Arquivos', color='yellow')
    pbar1.refresh()
    for item in lista:
        status_bar.update('Convertendo o arquivo: ' + str(item))
        df = pd.read_csv("zips/" + item, 
                         compression='zip',
                         header=None, 
                         sep=';', 
                         quotechar='"', 
                         encoding = "ISO-8859-1", 
                         low_memory=False,
                         dtype = value[3],
                         names=value[2])

        df.to_sql(name = value[0], con = connection, if_exists = 'append', index = False)
        pbar1.update()
    pbar.update()
    status_bar.update('Conversão Concluida!')
manager.stop()

CPU times: total: 14min 50s
Wall time: 48min 4s


In [14]:
# criar os indices corretos 
list_sql = [
    """CREATE INDEX IF NOT EXISTS IDX_SOCIOS ON SOCIOS (CNPJ ASC)""",
    """CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS ON ESTABELECIMENTOS (CNPJ ASC)""",
    """CREATE INDEX IF NOT EXISTS IDX_EMPRESAS ON EMPRESAS (CNPJ ASC)""",
    """ALTER TABLE ESTABELECIMENTOS ADD COLUMN CNPJ_FULL VARCHAR (14)""",
    """UPDATE ESTABELECIMENTOS set CNPJ_FULL = cnpj || cnpj_ordem || cnpj_dv""",
    """CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS_CNPJ_FULL ON ESTABELECIMENTOS (CNPJ_FULL ASC)"""
    
]
list_sql[1]


'CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS ON ESTABELECIMENTOS (CNPJ ASC)'

In [15]:
%%time
for i in list_sql:
    print(i)
    con.executescript(i)

CREATE INDEX IF NOT EXISTS IDX_SOCIOS ON SOCIOS (CNPJ ASC)
CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS ON ESTABELECIMENTOS (CNPJ ASC)
CREATE INDEX IF NOT EXISTS IDX_EMPRESAS ON EMPRESAS (CNPJ ASC)
ALTER TABLE ESTABELECIMENTOS ADD COLUMN CNPJ_FULL VARCHAR (14)
UPDATE ESTABELECIMENTOS set CNPJ_FULL = cnpj || cnpj_ordem || cnpj_dv
CREATE INDEX IF NOT EXISTS IDX_ESTABELECIMENTOS_CNPJ_FULL ON ESTABELECIMENTOS (CNPJ_FULL ASC)
CPU times: total: 2min 37s
Wall time: 39min 8s


In [16]:
%%time
df = pd.read_sql_query("SELECT count(CNPJ) from ESTABELECIMENTOS", connection)
con.close()
df.head()

CPU times: total: 594 ms
Wall time: 13.3 s


,count(CNPJ)
0,55155262


## Teste de encoding

In [5]:
encoding_list = ['ascii', 'big5', 'big5hkscs', 'cp037', 'cp273', 'cp424', 'cp437', 'cp500', 'cp720', 'cp737'
                 , 'cp775', 'cp850', 'cp852', 'cp855', 'cp856', 'cp857', 'cp858', 'cp860', 'cp861', 'cp862'
                 , 'cp863', 'cp864', 'cp865', 'cp866', 'cp869', 'cp874', 'cp875', 'cp932', 'cp949', 'cp950'
                 , 'cp1006', 'cp1026', 'cp1125', 'cp1140', 'cp1250', 'cp1251', 'cp1252', 'cp1253', 'cp1254'
                 , 'cp1255', 'cp1256', 'cp1257', 'cp1258', 'euc_jp', 'euc_jis_2004', 'euc_jisx0213', 'euc_kr'
                 , 'gb2312', 'gbk', 'gb18030', 'hz', 'iso2022_jp', 'iso2022_jp_1', 'iso2022_jp_2', 'iso2022_jp_2004'
                 , 'iso2022_jp_3', 'iso2022_jp_ext', 'iso2022_kr', 'latin_1','ISO-8859-1' 'iso8859_2'
                 , 'iso8859_3', 'iso8859_4', 'iso8859_5', 'iso8859_6', 'iso8859_7', 'iso8859_8', 'iso8859_9'
                 , 'iso8859_10', 'iso8859_11', 'iso8859_13', 'iso8859_14', 'iso8859_15', 'iso8859_16', 'johab'
                 , 'koi8_r', 'koi8_t', 'koi8_u', 'kz1048', 'mac_cyrillic', 'mac_greek', 'mac_iceland', 'mac_latin2'
                 , 'mac_roman', 'mac_turkish', 'ptcp154', 'shift_jis', 'shift_jis_2004', 'shift_jisx0213', 'utf_32'
                 , 'utf_32_be', 'utf_32_le', 'utf_16', 'utf_16_be', 'utf_16_le', 'utf_7', 'utf_8', 'utf_8_sig']

for encoding in encoding_list:
    worked = True
    try:
        df = pd.read_csv('zips/Cnaes.zip', 
                         compression='zip',
                         header=None, sep=';', 
                         quotechar='"', 
                         encoding=encoding)
    except:
        worked = False
    if worked:
        display(encoding, ':\n', df.head())

'cp037'

':\n'

,0
0,äÍ%ÈÑÎ?ÀÁ/ÊÊ?:äÍ%ÈÑÎ...


'cp273'

':\n'

...

':\n'

,0,1
0,111301,Cultivo de arroz
1,111302,Cultivo de milho
2,111303,Cultivo de trigo
3,111399,Cultivo de outros cereais n„o especificados an...
4,112101,Cultivo de algod„o herb·ceo


'ptcp154'

':\n'

,0,1
0,111301,Cultivo de arroz
1,111302,Cultivo de milho
2,111303,Cultivo de trigo
3,111399,Cultivo de outros cereais nгo especificados an...
4,112101,Cultivo de algodгo herbбceo


In [3]:
df = pd.read_csv('zips/Cnaes.zip', compression='zip',header=None, sep=';',quotechar='"', 
                         encoding = "ISO-8859-1", 
                         low_memory=False)
df

,0,1
0,111301,Cultivo de arroz
1,111302,Cultivo de milho
2,111303,Cultivo de trigo
3,111399,Cultivo de outros cereais não especificados an...
4,112101,Cultivo de algodão herbáceo
...,...,...
1354,9609208,Higiene e embelezamento de animais domésticos
1355,9609299,Outras atividades de serviços pessoais não esp...
1356,9700500,Serviços domésticos
1357,9900800,Organismos internacionais e outras instituiçõe...
